In [17]:
import pygame

import random
from collections import deque
import numpy as np
from helper import *

In [18]:
# Parameters
grid_size = 5  # Grid size for player and enemies (positions range from 0 to 4)
num_enemies = 2  # Number of enemies
visited_states = 2  # Visited status can be 0 or 1
wall_states = 2  # Wall status can be 0 (no wall) or 1 (wall)

movements = np.array([
    [0, 1],   # Move up
    [1, 0],   # Move right
    [-1, 0],  # Move left
    [0, -1]   # Move down
])

# Generate walls: 0 for no wall, 1 for wall

state_action_log = []


In [19]:
base_shape = [ visited_states, wall_states]
enemy_shape = [grid_size, grid_size] * num_enemies  # Each enemy has x and y positions
full_shape = base_shape + enemy_shape

# # Generate estimated_value_grid filled with zeros
estimated_value_grid = np.zeros(full_shape, dtype=float)

# # Generate random indices into the movements array
policy_indices = np.random.randint(0, 4, size=full_shape)

# # Create the policy array by indexing into movements
policy = movements[policy_indices]

# # Now, policy has shape full_shape + (2,), where the last dimension stores (dx, dy)
print("Policy Shape:", policy.shape)  # For verification

# # Generate reward_states: 40x40 grid of random 1s and 2s
reward_states = np.random.choice([1, 2], size=(grid_size, grid_size))

# # Output shapes for verification
print("Estimated Value Grid Shape:", estimated_value_grid.shape)
print("Reward States Shape:", reward_states.shape)

Policy Shape: (2, 2, 5, 5, 5, 5, 2)
Estimated Value Grid Shape: (2, 2, 5, 5, 5, 5)
Reward States Shape: (5, 5)


In [20]:
policy = {}
estimated_value_grid={}

In [21]:
def partition_maze_optimized(maze, s):
    x, y = s
    rows, cols = len(maze), len(maze[0])
    half_grid = 5 // 2

    partition = [
        [
            maze[(x + dx) % rows][(y + dy) % cols]
            for dy in range(-half_grid, half_grid + 1)
        ]
        for dx in range(-half_grid, half_grid + 1)
    ]
    return np.array(partition)

In [22]:
def optimized_relative_positions(maze_size, player_pos, enemy_positions, grid_size=5):
    x, y = player_pos
    rows, cols = maze_size  # Maze dimensions
    half_grid = grid_size // 2

    relative_positions = []

    for ex, ey in enemy_positions:
        # Compute differences considering wrap-around
        dx = (ex - x + cols) % cols
        if dx > cols // 2:
            dx -= cols

        dy = (ey - y + rows) % rows
        if dy > rows // 2:
            dy -= rows

        # Check if enemy is within the local grid
        if -half_grid <= dx <= half_grid and -half_grid <= dy <= half_grid:
            # Map to local grid coordinates (0 to 4)
            local_x = int(dx + half_grid)
            local_y = int(dy + half_grid)
            relative_positions.append((local_x, local_y))
    return relative_positions

In [23]:
def check_collision(current_state, enemies):
    x, y = current_state
    for enemy in enemies:
        ex, ey = enemy["pos"]
        if (x, y) == (ex, ey):
            return True
    return False


In [24]:
def is_wall_ahead(current_state, action, reward_maze):
    x, y = current_state
    dx, dy = action
    maze_rows, maze_cols = len(reward_maze),len(reward_maze[0])
    nextx, nexty = (x + dx) % maze_rows, (y + dy) % maze_cols
    return reward_maze[nextx][nexty] == 1  # Returns True if the next cell is a wall


In [25]:
def get_valid_actions(current_state, reward_maze):
    x, y = current_state
    maze_rows, maze_cols = len(reward_maze),len(reward_maze[0])
    valid_actions = []
    for dx, dy in movements:
        nextx, nexty = (x + dx) % maze_rows, (y + dy) % maze_cols
        if reward_maze[nextx][nexty] != 1:  # Not a wall
            valid_actions.append((dx, dy))
    return valid_actions


In [26]:
def Next_action(current_state, policy, enemies, reward_maze, grid_size=5):
    maze_rows, maze_cols = len(reward_maze),len(reward_maze[0])
    x, y = current_state

    # Retrieve enemy positions and compute relative positions
    enemy_positions = [enemy["pos"] for enemy in enemies]
    enemy_positions = optimized_relative_positions(
        (maze_rows, maze_cols), current_state, enemy_positions, grid_size=grid_size
    )

    # Get food and wall status at the current position
    food = int(reward_maze[x][y] == 2)
    wall = int(reward_maze[x][y] == 1)

    # Prepare enemy positions for the state index
    max_enemies_in_local_grid = 2
    enemy_positions.sort()
    padded_enemy_positions = enemy_positions[:max_enemies_in_local_grid]
    num_missing = max_enemies_in_local_grid - len(padded_enemy_positions)
    padded_enemy_positions.extend([(-1, -1)] * num_missing)  # Use placeholder (-1, -1)

    # Flatten the enemy positions list
    enemy_positions_flat = [coord for pos in padded_enemy_positions for coord in pos]

    # Build the state index tuple
    indices_s = (food, wall) + tuple(enemy_positions_flat)

    # Retrieve the action from the policy dictionary
    action = policy.get(indices_s)

    # If action is None or leads into a wall, select a valid action
    if action is None or is_wall_ahead(current_state, action, reward_maze):
        valid_actions = get_valid_actions(current_state, reward_maze)
        if valid_actions:
            action = random.choice(valid_actions)
        else:
            action = (0, 0)  # No valid moves, stay in place

    # Append the state index, action, reward value, and current state to the log
    state_action_log.append((indices_s, action, reward_maze[x][y], current_state))
    return action


In [27]:
def G(estimated_value_grid, enemies, reward_maze, gamma=0.85):
    G = 0  # Initialize the return
    returns = []  # List to store the returns for each state-action pair

    # Dictionaries to store cumulative sums and counts
    returns_sum = {}
    returns_count = {}

    # Iterate over the episode in reverse
    for entry in reversed(state_action_log):
        indices_s, action, reward_value, current_state = entry
        x, y = current_state
        dx, dy = action

        # Check for collision with enemies using the true state
        collision = check_collision(current_state, enemies)
        wall_ahead = is_wall_ahead(current_state, action, reward_maze)

        if collision:
            reward = -100  # Collision with enemy
        elif wall_ahead:
            reward = -10 # Attempted to move into a wall
        elif reward_value == 2:  # Food
            reward = 2
        else:
            reward = -1  # Default penalty for empty space

        # Update the return
        G = reward + gamma * G

        # Store the return with the corresponding state-action pair
        state_action_pair = (indices_s, tuple(action))
        returns.append((state_action_pair, G))

    # Reverse the returns list to match the original order
    returns.reverse()

    # Update the estimated value grid
    for state_action_pair, G in returns:
        indices_s, action = state_action_pair

        # Initialize if not already in dictionaries
        if state_action_pair not in returns_sum:
            returns_sum[state_action_pair] = 0
            returns_count[state_action_pair] = 0

        # Update cumulative sums and counts
        returns_sum[state_action_pair] += G
        returns_count[state_action_pair] += 1

        # Compute the average return
        average_return = returns_sum[state_action_pair] / returns_count[state_action_pair]

        # Update the estimated value grid
        estimated_value_grid[state_action_pair] = average_return

    return estimated_value_grid


In [28]:
def update_policy(policy, estimated_value_grid, movements, epsilon=0.2):
    # Create a set of unique states from the state_action_log
    unique_states = set(entry[0] for entry in state_action_log)

    for indices_s in unique_states:
        max_value = float('-inf')
        best_action = None

        for action in movements:
            action_tuple = tuple(action)
            state_action_pair = (indices_s, action_tuple)


            # Retrieve the estimated value of the state-action pair
            estimated_value = estimated_value_grid.get(state_action_pair, 0)


            if estimated_value > max_value:
                max_value = estimated_value
                best_action = action

        # Epsilon-greedy policy update
        if random.random() <= epsilon:
            # Explore: choose a random action
            policy[indices_s] = random.choice(movements)
        else:
            # Exploit: choose the best action
            policy[indices_s] = best_action

    return policy


In [29]:
random_value = random.random()
print(random_value)

0.9993585880262903


In [30]:
def Next_Cycle(policy,estimated_value_grid, enemies, reward_maze, movements,grid_size=5,discount=0.9):
    estimated_value_grid=G(estimated_value_grid, enemies, reward_maze,discount)
    policy=update_policy(policy, estimated_value_grid, movements, epsilon=0.1)
    state_action_log=[]
    return policy,state_action_log

In [31]:
def initialize_positions(maze, num_enemies):
    maze_rows, maze_cols = len(maze), len(maze[0])

    # Find all possible positions (excluding walls)
    possible_positions = [(x, y) for x in range(maze_rows) for y in range(maze_cols) if maze[x][y] != 1]

    # Randomly select a position for the player
    player_pos = random.choice(possible_positions)

    # Remove player's position from possible positions
    possible_positions.remove(player_pos)

    enemies = []
    for _ in range(num_enemies):
        if not possible_positions:
            break  # No more positions available
        enemy_pos = random.choice(possible_positions)
        enemies.append({"pos": enemy_pos, "target": None})
        possible_positions.remove(enemy_pos)

    return player_pos, enemies


In [32]:
pygame.init()
maxt=50
def main():
    global policy,maxt
    timeout=maxt
    """Main game loop."""
    clock = pygame.time.Clock()
    running = True

    # Store the maze to prevent re-generation
    static_maze = create_maze(ROWS, COLS)

    # Initialize the game
    def restart_game():
        
        maze = create_maze(ROWS, COLS)
        player_pos, enemies = initialize_positions(maze, num_enemies)
        # Use the pre-generated static maze
        return maze, player_pos, enemies, 0

    # Initialize the maze, player, enemies, and score
    maze, player_pos, enemies, score = restart_game()

# Main game loop
    while running:
        # Handle events (e.g., quitting the game)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Handle player movement
        if timeout==0:
            policy,state_action_log=Next_Cycle(policy,estimated_value_grid, enemies, maze, movements,grid_size=5,discount=0.9)
            if maxt<10000:
                maxt+=100
            timeout=maxt
            maze, player_pos, enemies, score = restart_game()
        current_state = player_pos
        action = Next_action(current_state,policy,enemies,maze,grid_size=5)
        timeout-=1
        print(timeout)
        new_player_pos = move_entity(player_pos, action)
        if is_valid_move(new_player_pos, maze):
            if maze[new_player_pos[0]][new_player_pos[1]] == 2:  # Collect food
                score += 1
                maze[new_player_pos[0]][new_player_pos[1]] = 0
            player_pos = new_player_pos

        # Check for collisions
        for enemy in enemies:
            if enemy["pos"] == player_pos or check_collision(current_state, enemies):
                print("Game Over! Restarting...")
                if maxt<10000:
                    maxt+=100
                timeout=maxt
                policy,state_action_log=Next_Cycle(policy,estimated_value_grid, enemies, maze, movements,grid_size=5,discount=0.9)
                maze, player_pos, enemies, score = restart_game()

        # Check if all food is collected
        if all(maze[row][col] != 2 for row in range(ROWS) for col in range(COLS)):
            print("You Win! Restarting...")
            timeout=maxt
            policy,state_action_log=Next_Cycle(policy,estimated_value_grid, enemies, maze, movements,grid_size=5,discount=0.9)
            maze, player_pos, enemies, score = restart_game()
        move_enemies(enemies, maze)
        draw_maze(maze, player_pos, enemies)
        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()

if __name__ == "__main__":
    main()


49
Enemy at (38, 36) assigned new target (22, 15)
Enemy moved to (37, 36)
Enemy at (36, 9) assigned new target (22, 15)
Enemy moved to (35, 9)
48
Enemy moved to (36, 36)
Enemy moved to (34, 9)
47
Enemy moved to (35, 36)
Enemy moved to (33, 9)
46
Enemy moved to (35, 37)
Enemy moved to (32, 9)
45
Enemy moved to (35, 38)
Enemy moved to (31, 9)
44
Enemy moved to (34, 38)
Enemy moved to (30, 9)
43
Enemy moved to (33, 38)
Enemy moved to (29, 9)
42
Enemy moved to (32, 38)
Enemy moved to (28, 9)
41
Enemy moved to (31, 38)
Enemy moved to (27, 9)
40
Enemy moved to (30, 38)
Enemy moved to (26, 9)
39
Enemy moved to (29, 38)
Enemy moved to (25, 9)
38
Enemy moved to (28, 38)
Enemy moved to (24, 9)
37
Enemy moved to (27, 38)
Enemy moved to (23, 9)
36
Enemy moved to (27, 39)
Enemy moved to (22, 9)
35
Enemy moved to (26, 39)
Enemy moved to (22, 10)
34
Enemy moved to (25, 39)
Enemy moved to (22, 11)
33
Enemy moved to (24, 39)
Enemy moved to (22, 12)
32
Enemy moved to (23, 39)
Enemy moved to (22, 13)
31
